In [2]:
!git clone https://github.com/AnupJoseph/topic_centrality.git

fatal: destination path 'topic_centrality' already exists and is not an empty directory.


In [ ]:
!pip install textacy

In [20]:
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null --fix-missing
!wget https://mirrors.estointernet.in/apache/spark/spark-3.0.1/spark-3.0.1-bin-hadoop2.7.tgz
!tar xvf spark-3.0.1-bin-hadoop2.7.tgz
!pip install -q findspark
!pip install pyLDAvis textacy -qq > /dev/null

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:8 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:9 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Ign:13 https://developer.download.nvidia.com/c

In [21]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.1-bin-hadoop2.7"
import findspark
findspark.init()
from pyspark.sql import SparkSession
sc = SparkSession.builder.master("local[*]").getOrCreate()

In [14]:
import pandas as pd
import numpy as np

import bs4
# import 
from textacy.preprocessing.replace import replace_urls
# %time text = replace_urls(long_s,replace_with= '')

In [22]:
from pyspark.sql.types import *
import pyspark.sql.functions as func

# gensim
import gensim
# from gensim.utils import simple_preprocess
# from gensim.parsing.preprocessing import STOPWORDS
from gensim.models.coherencemodel import CoherenceModel
from gensim.test.utils import datapath
from gensim import corpora, models

# general
import re

In [3]:
COLS = ['id', 'created_at', 'original_text','clean_text', 'retweet_count', 'hashtags','mentions', 'original_author']
politicians = ['SenSanders','realDonaldTrump', 'JoeBiden', 'andrewcuomo', 'TeamPelosi', 'NikkiHaley', 'MittRomney', 'Mike_Pence','SenatorCollins','PeteButtigieg']
RETWEET_COLS = ['original_tweet_id','retweet_id','type','created_at','source','favorite_count','retweet_count','original_author']

In [15]:
def processor(text):
  text = replace_urls(text,replace_with= '')
  text = bs4.BeautifulSoup(text,'html.parser').get_text()
  # text = replace_urls(text,replace_with= '')
  return text

In [18]:
for politician in politicians:
  outdata = pd.DataFrame(columns=COLS)
  data = pd.read_csv(f'topic_centrality/data/{politician}/{politician}_data.csv')
  data.replace(np.nan,'',inplace=True)
  data['clean_text'] = data['clean_text'].apply(processor)
  data.to_csv(f'topic_centrality/data/{politician}/{politician}_data.csv',header=True)

In [ ]:
# Build the final spark dataframe for analysis below

In [25]:
final_data = sc.read.format("csv").options(header='true', inferschema='true').load(os.path.realpath("topic_centrality/data/SenSanders/SenSanders_data.csv"))

In [26]:
for politician in politicians[1:]:
  next_frame = sc.read.format("csv").options(header='true', inferschema='true').load(os.path.realpath(f"topic_centrality/data/{politician}/{politician}_data.csv"))
  final_data = final_data.union(next_frame)

In [27]:
final_data.show()

+---+----------+-------------------+--------------------+--------------------+--------------------+-------------+--------------------+--------+---------------+
|_c0|Unnamed: 0|                 id|          created_at|       original_text|          clean_text|retweet_count|            hashtags|mentions|original_author|
+---+----------+-------------------+--------------------+--------------------+--------------------+-------------+--------------------+--------+---------------+
|  0|         0|1294257465412870144|2020-08-14 13:00:...|Grocery store wor...|grocery store wor...|         1423|               <UNK>|   <UNK>|     SenSanders|
|  1|         1|1294050558337339394|2020-08-13 23:17:...|During the worst ...|bad economic cris...|         4895|               <UNK>|   <UNK>|     SenSanders|
|  2|         2|1293996805110333440|2020-08-13 19:44:...|Black women must ...|black woman equal...|         2391|#BlackWomensEqual...|   <UNK>|     SenSanders|
|  3|         3|1293961810719244288|2020

In [28]:
tweets = final_data.rdd.map(lambda x : x["clean_text"]).filter(lambda x: x is not None)
tweets = tweets.map( lambda document: re.split(" ", document)).zipWithIndex()

In [37]:
processed_docs = tweets.map(lambda x: x[0]).collect()
word_dict = gensim.corpora.Dictionary(processed_docs)

word_dict.filter_extremes(no_below=4, no_above=0.8, keep_n=10000)
bow_corpus = [word_dict.doc2bow(doc) for doc in processed_docs]

In [30]:
bow_doc_1 = bow_corpus[0]
print("Showing bag of words for doc 0")

for i in range(len(bow_doc_1)):
  print(f"Word {bow_doc_1[i][0]} (\"{dict[bow_doc_1[i][0]]}\") appears {bow_doc_1[i][1]} time.")

Showing bag of words for doc 0
Word 0 ("corporation") appears 1 time.
Word 1 ("deserve") appears 2 time.
Word 2 ("exploit") appears 1 time.
Word 3 ("gear") appears 1 time.
Word 4 ("grocery") appears 1 time.
Word 5 ("hazard") appears 1 time.
Word 6 ("hero") appears 1 time.
Word 7 ("life") appears 1 time.
Word 8 ("line") appears 1 time.
Word 9 ("need") appears 1 time.
Word 10 ("pandemic") appears 1 time.
Word 11 ("protective") appears 1 time.
Word 12 ("store") appears 1 time.
Word 13 ("worker") appears 1 time.


In [31]:
ldaModel = gensim.models.LdaMulticore(
  bow_corpus,
  num_topics=7,
  id2word=dict,
  alpha=0.01,
  eta=0.5,
  offset=8,
  random_state=100,
  chunksize=100,
  passes=1,
  per_word_topics=True
)

In [32]:
# Compute Perplexity
print('LDA Perplexity: ', ldaModel.log_perplexity(bow_corpus)) # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherenceModelLda = CoherenceModel(model=ldaModel, texts=processed_docs, dictionary=dict, coherence='c_v')
coherenceLda = coherenceModelLda.get_coherence()
print('LDA Coherence Score: ', coherenceLda)

LDA Perplexity:  -8.301671531331511
LDA Coherence Score:  0.3332648970508248


In [33]:

'''
For each topic, we will explore the words occuring in that topic and its relative weight
'''
for idx, topic in ldaModel.print_topics(-1):
    print("Topic: {} \nWords: {}".format(idx, topic ))
    print("\n")

Topic: 0 
Words: 0.019*"support" + 0.019*"join" + 0.016*"Iowa" + 0.015*"DemDebate" + 0.014*"today" + 0.013*"tonight" + 0.013*"chip" + 0.012*"serve" + 0.012*"vote" + 0.010*"ready"


Topic: 1 
Words: 0.024*"president" + 0.019*"Trump" + 0.015*"year" + 0.015*"election" + 0.014*"page" + 0.013*"office" + 0.013*"know" + 0.013*"DemDebate" + 0.013*"Donald" + 0.012*"american"


Topic: 2 
Words: 0.058*"hope" + 0.041*"turn" + 0.016*"story" + 0.012*"chance" + 0.010*"time" + 0.009*"give" + 0.008*"well" + 0.008*"país" + 0.008*"page" + 0.007*"learn"


Topic: 3 
Words: 0.015*"day" + 0.010*"trust" + 0.008*"United" + 0.008*"people" + 0.008*"China" + 0.007*"state" + 0.007*"States" + 0.007*"America" + 0.007*"freedom" + 0.007*"President"


Topic: 4 
Words: 0.047*"thank" + 0.018*"service" + 0.017*"today" + 0.014*"great" + 0.012*"year" + 0.012*"South" + 0.012*"come" + 0.011*"look" + 0.010*"country" + 0.009*"forward"


Topic: 5 
Words: 0.014*"reform" + 0.014*"small" + 0.013*"violence" + 0.013*"start" + 0.012*"

In [34]:
import pyLDAvis.gensim
pyLDAvis.enable_notebook()

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
data = pyLDAvis.gensim.prepare(ldaModel, bow_corpus, dict, mds='mmds')
data

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
6      0.005266 -0.271881       1        1  31.241713
3     -0.222948  0.068883       2        1  15.618894
1      0.177952  0.212865       3        1  13.679408
4      0.281527 -0.079420       4        1  12.733537
0      0.095298 -0.076008       5        1  12.114270
5     -0.250302 -0.137169       6        1   8.319190
2     -0.086793  0.282730       7        1   6.292989, topic_info=           Term         Freq        Total Category  logprob  loglift
509       thank  1439.000000  1439.000000  Default  30.0000  30.0000
1101       hope   876.000000   876.000000  Default  29.0000  29.0000
467        turn   630.000000   630.000000  Default  28.0000  28.0000
95    president  1875.000000  1875.000000  Default  27.0000  27.0000
67        Trump   697.000000   697.000000  Default  26.0000  26.0000
...         ...          ...          ...      ...      ...      ...
1531       road    66.907422   119.769645   Topic7  -5.4104   2.1835
384       place    71.156444   207.841678   Topic7  -5.3489   1.6938
267        know    85.804355   792.034702   Topic7  -5.1617   0.5432
1392       find    68.130620   257.561106   Topic7  -5.3923   1.4359
40        woman    71.822207   742.068995   Topic7  -5.3396   0.4305

[434 rows x 6 columns], token_table=      Topic      Freq       Term
term                            
307       2  0.009163           
307       3  0.009163           
307       6  0.971288           
1318      2  0.022445         ""
1318      3  0.022445         ""
...     ...       ...        ...
162       3  0.007008  yesterday
162       4  0.021024  yesterday
162       5  0.007008  yesterday
162       6  0.007008  yesterday
162       7  0.007008  yesterday

[2167 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[7, 4, 2, 5, 1, 6, 3])

In [35]:
pyLDAvis.save_html(data,'ldaviz.html')

In [42]:
def predict(new_doc,lda_model,word_dict):
    try:
        new_doc = new_doc.split()
        BoW = word_dict.doc2bow(new_doc)
        doc_topics, _, _ = lda_model.get_document_topics(BoW, per_word_topics=True)
        return sorted(doc_topics,key=lambda x: x[1], reverse=True)[0][0]
    except:
        # Some rows may have null clean text (example: every token in the tweet is <3 character long). If that's the case return -1 (we want to discard these)
        return -1

def add_cluster(username,lda_model,word_dict):
    """
    Addeds the LDA clusters to a users twitter timeline data frame.
    Parameters
    ----------
    :param username: The twitter handle of the user being modelled.
    
    :param lda_model: Some gensim LDA model.
    
    :param word_dict: a dictionary coresponding that maps elements of the corpus to words.
    """
    file_path = f"topic_centrality/data/{username}/{username}_data.csv"
    timeline_df = pd.read_csv(file_path)
    timeline_df["lda_cluster"] = timeline_df["clean_text"].apply(lambda x : predict(x,lda_model,word_dict))
    csvFile = open(file_path, 'w' ,encoding='utf-8')
    timeline_df.to_csv(csvFile, mode='w', index=False, encoding="utf-8")
    print(f"Saved {username}")

In [43]:
for politician in politicians:
    add_cluster(politician,ldaModel,word_dict)

Saved SenSanders
Saved realDonaldTrump
Saved JoeBiden
Saved andrewcuomo
Saved TeamPelosi
Saved NikkiHaley
Saved MittRomney
Saved Mike_Pence
Saved SenatorCollins
Saved PeteButtigieg


In [44]:
import sys
import tqdm
import numpy as np
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
from networkx.drawing.nx_agraph import graphviz_layout

# def return_colour(aNum):
#     colours = ["#006816","#8d34e4","#c9a738","#0163d0","#ee5700", "#00937e", "#ff4284", "#4b5400", "#ea80ff","#9f0040"]
#     assert aNum < len(colours)
#     return colours[aNum], aNum+1

# def return_legend(legend):
#     legends = [Line2D([0], [0], marker='o', color='w', label='Party Leader', markerfacecolor='r', markersize=10), Line2D([0], [0], marker='o', color='w', label='Retweet', markerfacecolor='#79BFD3', markersize=10)]
#     legend = sorted(legend, key=lambda tup: tup[1])
#     for color, cluster_num in legend:
#         legends.append(Line2D([0], [0], marker='o', color='w', label='Topic {}'.format(cluster_num), markerfacecolor=color, markersize=10))
#     return legends

class Graph(object):
    '''
    Initiates the networkx graph, houses visualization, as well as some quick/dirty analysis.
    Parameters
    ----------
    :param usernames: A list of strings, corresponding to the twitter usernames stored in `/data`
    
    :param n: An int, corresponding to the number of tweets to map.
    '''
    def __init__(self, usernames, n=None):
        self.num_retweeters = 0
        self.num_tweets = 0
        self.num_retweets = 0
        self.G = nx.Graph()
        self.title = ""
        for username in usernames:
            self.title += "{}_".format(username)
            user_graph = self.build_graph(username, n)
            self.G = nx.compose(self.G, user_graph)
        print("--- {} tweets, {} retweeters, {} retweets ---".format(self.num_tweets,self.num_retweeters, self.num_retweets))

    def draw_graph(self, G=None, save=False, file_type='png',use_pos=False):
        """
        Handles rendering and drawing the network.
        Parameters
        ----------
        :param G: `optional` 
        
        a networkx graph. If present draws this graph instead of the one built in the constructor.
        :param save: `optional` 
        
        A boolean. If true saves an image of the graph to `/visualizations` otherwise renders the graph.
        
        :param file_type: `optional` 
        
        A string. If save flag is true it saves graph with this file extension.
        
        :param use_pos: `optional` 
        
        A boolean. If true renders the graph using default positions of the entire graph. Otherwise calculates positions based on data used.
        """
        if not G:
            G = self.G
        print("--- Adding colours and labels ---")
        colors = []
        legend = set()
        labels = {}
        pbar = tqdm.tqdm(total=len(G.nodes()))
        for node in G.nodes():
            attributes = G.nodes[node]
            assert 'type' in attributes, "Type is a required attribute for any vertex in this graph."
            if attributes['type'] == 'retweet':
                colors.append('#79BFD3')
            elif attributes['type'] == 'tweet':
                cluster = self.__return_colour(attributes["lda_cluster"])
                legend.add(cluster)
                colors.append(cluster[0])
            elif attributes['type'] == 'user':
                labels[node] = node
                colors.append('red')
            pbar.update(1)
        pbar.close()
        plt.figure(figsize=(30, 30))
        pos = graphviz_layout(self.G, prog="sfdp") if use_pos else graphviz_layout(G, prog="sfdp")
        _xlim = None
        _ylim = None
        if use_pos: #To use the same positioning for two subgraphs we need to make sure the x/y limits are the same for the plot
            nx.draw(self.G,pos)
            _xlim = plt.gca().get_xlim() # grab the xlims
            _ylim = plt.gca().get_ylim() # grab the ylims
            # Clear the figure.
            plt.clf()
        print("--- Drawing {} nodes and {} edges ---".format(len(G.nodes()), G.number_of_edges()))
        nx.draw(G, pos,
                node_color=colors,
                with_labels=False,
                alpha=0.75,
                node_size=20,
                width=0.2,
                arrows=False
                )
        if use_pos: plt.axis( [ _xlim[0], _xlim[1], _ylim[0], _ylim[1] ] )
        nx.draw_networkx_labels(G, pos, labels, font_size=16, font_color='r')
        plt.legend(handles=self.__return_legend(legend), loc="best")
        topics_used = '_'.join(str(l[1]) for l in sorted(legend, key=lambda tup: tup[1]))
        plt.savefig("../visualizations/{}graph_{}_tweets_{}_retweeters_{}_retweets_topics{}.{}".format(self.title,self.num_tweets, self.num_retweeters, self.num_retweets,topics_used, file_type)) if save else plt.show()

    def __return_legend(self, legend):
        return return_legend(legend)

    def build_graph(self, username, n=None):
        twitter_df = pd.read_csv(f"topic_centrality/data/{username}/{username}_data.csv")
        if n:
            twitter_df = twitter_df.sample(n=min(n, len(twitter_df)),random_state=1)
        # twitter_df = twitter_df.set_index('id')
        retweet_df = pd.read_csv(f"topic_centrality/data/{username}/{username}_retweets.csv")
        # if we're only taking 20 tweets find all the retweets for those 20
        retweet_df = retweet_df[retweet_df['original_tweet_id'].isin(
            twitter_df['id'])]
        # Instantiate a new Graph (there could potentially be multiple edges between a pair of nodes)
        G = nx.Graph()
        G.add_node(username, type='user')
        # add tweet nodes
        nodes = twitter_df.drop_duplicates(subset="id").set_index('id').to_dict('index').items()
        G.add_nodes_from(nodes)
        print("--- adding edges ---")
        pbar = tqdm.tqdm(total=len(twitter_df)+len(retweet_df))
        for _, row in twitter_df.iterrows():
            pbar.update(1)
            G.add_edge(username, row['id'])
        # add retweet user nodes (those who retweeted the original tweets) multipl
        user_nodes = retweet_df.drop_duplicates(subset="original_author")
        user_nodes = user_nodes.set_index('original_author').to_dict('index').items()
        G.add_nodes_from(user_nodes)
        for _, row in retweet_df.iterrows():
            pbar.update(1)
            G.add_edge(row['original_tweet_id'], row['original_author'])
        pbar.close()
        self.num_retweeters += len(user_nodes)
        self.num_tweets += len(twitter_df)
        self.num_retweets += len(retweet_df)
        return G

    # def get_density(self):
    #     density = nx.density(self.G)
    #     print("The percentage of edges/possible edges is {0:.4f}%: ".format(density*100))
    #     return density

    def map_topics(self, topics):
        '''
        Rebuilds the graph with only certain tweet topics
        Parameters
        ----------
        :param topics: A list of ints between 0-k, where k is the number of topics-1, corresponding with the topics to isolate for
        '''
        mapped_graph = self.G.copy()
        remove = [node for node in mapped_graph.nodes() if "lda_cluster" in mapped_graph.nodes[node] and not mapped_graph.nodes[node]["lda_cluster"] in topics]
        mapped_graph.remove_nodes_from(remove)
        remove = [node for node in mapped_graph.nodes() if "type" in mapped_graph.nodes[node] and mapped_graph.nodes[node]["type"] == "retweet" and mapped_graph.degree(node) == 0]
        mapped_graph.remove_nodes_from(remove)        
        assert not any(node in remove for node in mapped_graph.nodes)
        return mapped_graph

    # def __return_colour(self, aNum):
    #     return return_colour(aNum)
        
    # def diameter(self,G=None):
    #     if not G:
    #         G = self.G
    #     d = nx.diameter(G)
    #     print("Diameter: {}".format(d))
    #     return d

    # def retweet_histogram(self):
    #     degree_sequence = [d for n, d in self.G.degree() if self.G.nodes[n]["type"] == "retweet"]
    #     return np.histogram(degree_sequence, bins=len(degree_sequence)//2)

    def __len__(self):
        return len(self.G)

if __name__ == '__main__':
    # Read in CSV file for that twitter user (these are the original tweets)
    usernames = [politicians[0],politicians[1]]
    topics = range(0,8)
    G = Graph(usernames,100)
    # print(G.retweet_histogram())
    # G.draw_graph(save=True)
    # G.draw_graph(save=True,use_pos=True)
    # for i in topics:
    #     removed = G.map_topics([i])
    #     G.draw_graph(G=removed,save=True,use_pos=True)

 12%|█▏        | 917/7894 [00:00<00:00, 9165.81it/s]

--- adding edges ---


 21%|██        | 921/4487 [00:00<00:00, 9206.45it/s]

--- adding edges ---


100%|██████████| 4487/4487 [00:00<00:00, 9845.88it/s]

--- 200 tweets, 2597 retweeters, 12181 retweets ---


In [50]:
import sys
# from build_graph import Graph
import networkx as nx
import matplotlib.pyplot as plt
from numpy import log10
import numpy as np

NUM_TOPICS = 7
COLOURS = ["#006816", "#8d34e4", "#c9a738", "#0163d0", "#ee5700", "#00937e", "#ff4284", "#4b5400", "#ea80ff", "#9f0040"]

def centrality_per_topic(G, username=None, measure='mean'):
    """
        Calculates the eigenvector centrality for a network G, and then takes the sums/averages the centrality of tweets grouped by topic. 
        Parameters
        ----------
        :param G: The networkx Graph to calculate centrality from
        :param username: `optional` if present only uses that user's tweets when calculating the mean.
        :param measure: `optional` Options are ('mean','sum','zscore'). Choooses how to represent the aggregate zscores
    """
    assert (measure in ["mean","sum","zscore"]), "Unknown measure used, must be one of mean, sum, or zscore."
    netx_graph = G.copy()
    # centrality = nx.eigenvector_centrality_numpy(netx_graph)
    centrality = nx.eigenvector_centrality_numpy(netx_graph)
    # (topic, []) for topic in range(NUM_TOPICS)
    topic_centrality = {}
    for topic in range(NUM_TOPICS):
      topic_centrality[topic] = []
    for node in netx_graph.nodes():
        node_attributes = netx_graph.nodes[node]
        node_centrality = centrality[node]
        if "lda_cluster" in node_attributes:
            if username and username in netx_graph.neighbors(node) or username is None:
                topic = node_attributes["lda_cluster"]
                if topic in topic_centrality:
                    topic_centrality[topic].append(node_centrality)
                else:
                    topic_centrality[topic] = [node_centrality]
    all_topic_centralities = np.concatenate([centrality for _, centrality in topic_centrality.items()]).ravel()
    average_centrality = np.mean(all_topic_centralities)
    std_dev = np.std(all_topic_centralities)
    for topic, centrality_list in topic_centrality.items():
        if measure == 'zscore':
            # Calculate the z-score for each tweet of topic i relative to the average tweet centrality for the entire graph (or subgraph give).
            zscores = (np.array(centrality_list)-average_centrality)/std_dev
            # Calcualte the mean of the z-scores.
            topic_centrality[topic] = np.mean(zscores)
        elif measure == 'sum':
            topic_centrality[topic] = np.sum(centrality_list)
        else:
            topic_centrality[topic] = np.mean(centrality_list)
    return topic_centrality

def plot_dual_centralities(overall_net_cent, individaul_cents, expand=False, usernames=None, measure='mean'):
    """
        Plots the "overall network centrality" (the centralities of every tweet of a certain topic) for each topic over the 
        "individual network centrality" (the centralities of every tweet of a certain topic for a certain user)
        Parameters
        ----------
        :param overall_net_cent: A dictionary where each key is a topic, and the value is the sum/mean centrality for every tweet in the network of that topic. 
        :param individaul_cents: A list of dictionaries, where each element contains the centrality dictionary for each topic tweeted about by that user.
        :param expand: `optional` If true it plots all of the different users as with their individual centrality scores. Otherwise means all of them per topic.
        :param usernames: `optional` If expand is true this should correspond to the order of usernames in the individual_cents list. For plot legend purposes.
        :param measure: `optional` Options are ('mean','sum','zscore'). For title/filename purposes.
    """
    assert (measure in ["mean","sum","zscore"]), "Unknown measure used, must be one of mean, sum, or zscore."
    overall_net_cent = sorted(overall_net_cent.items(), key=lambda tup: tup[0])
    # get rid of topic -1 (empty tweets)
    overall_net_cent = overall_net_cent[1:] if overall_net_cent[0][0] == -1 else overall_net_cent
    leader_cents = []
    for leader_cent in individaul_cents:
        leader_cent = sorted(leader_cent.items(), key=lambda tup: tup[0])
        # get rid of topic -1 (empty tweet)
        leader_cent = leader_cent[1:] if leader_cent[0][0] == - \
            1 else leader_cent
        topics, centralities = map(list, zip(*leader_cent))
        centralities = np.nan_to_num(np.array(centralities))
        leader_cents.append(centralities)
    topics, average_overall_cents = map(list, zip(*overall_net_cent))
    if measure == "zscore":
        ax = plt.gca()
        ax.spines['top'].set_color('none')
        ax.spines['bottom'].set_position('zero')
        ax.spines['left'].set_position('zero')
        ax.spines['right'].set_color('none')
        ax.axis(xmin=-1, xmax=1, ymin=-1, ymax=1)
        ax.set_xlabel('Topic Centrality for Individual\'s Network ({})'.format(measure), labelpad=125)
        ax.set_ylabel('Overall Network Topic Centrality ({})'.format(measure), labelpad=165)
    else:
        plt.xlabel('Topic Centrality for Individual\'s Network ({})'.format(measure))
        plt.ylabel('Overall Network Topic Centrality ({})'.format(measure))
    plt.title('Overall Importance of Topic/Importance of Topic to Individual ({})\n'.format(measure))
    plt.grid(True)
    if expand and usernames:
        assert len(usernames) == len(leader_cents)
        for i, (username, centralities) in enumerate(zip(usernames, leader_cents)):
            plt.scatter(centralities, average_overall_cents,
                        alpha=0.5, label=username, c=COLOURS[i])
            for j, txt in enumerate(topics):
                plt.annotate("{}".format(txt+1),
                             (centralities[j], average_overall_cents[j]))
            plt.legend()
        plt.savefig(
            "{}_opposing_centrality_chart_(expanded).png".format(measure))
        plt.clf()
    else:
        average_individual_cents = np.mean(leader_cents, axis=0)
        plt.scatter(average_individual_cents, average_overall_cents, alpha=0.5)
        for j, txt in enumerate(topics):
            plt.annotate("Topic {}".format(txt+1), (average_individual_cents[j], average_overall_cents[j]))
        plt.savefig("{}_opposing_centrality_chart_(mean_of_leaders).png".format(measure))
        plt.clf()

if __name__ == "__main__":
    usernames = [politicians[0],politicians[1]]
    G = Graph(usernames,100).G
    sum_overall_topic_centralities = centrality_per_topic(G, measure='sum')
    mean_overall_topic_centralities = centrality_per_topic(G, measure='mean')
    zscore_overall_topic_centralities = centrality_per_topic(G, measure='zscore')
    sum_leader_cents = []
    mean_leader_cents = []
    zscore_leader_cents = []
    for username in usernames:
        single_leader_g = Graph([username],100).G
        sum_leader_cents.append(centrality_per_topic(single_leader_g, measure='sum'))
        mean_leader_cents.append(centrality_per_topic(single_leader_g, measure='mean'))
        zscore_leader_cents.append(centrality_per_topic(single_leader_g, measure='zscore'))
    plot_dual_centralities(sum_overall_topic_centralities,
                           sum_leader_cents, expand=False, usernames=usernames, measure='sum')
    plot_dual_centralities(sum_overall_topic_centralities,
                           sum_leader_cents, expand=True, usernames=usernames, measure='sum')
    plot_dual_centralities(mean_overall_topic_centralities,
                           mean_leader_cents, expand=False, usernames=usernames, measure='mean')
    plot_dual_centralities(mean_overall_topic_centralities,
                           mean_leader_cents, expand=True, usernames=usernames, measure='mean')
    plot_dual_centralities(zscore_overall_topic_centralities,
                           zscore_leader_cents, expand=False, usernames=usernames, measure='zscore')
    plot_dual_centralities(zscore_overall_topic_centralities,
                           zscore_leader_cents, expand=True, usernames=usernames, measure='zscore')


 23%|██▎       | 1852/7894 [00:00<00:00, 8975.83it/s]

--- adding edges ---


 20%|██        | 909/4487 [00:00<00:00, 9088.49it/s]

--- adding edges ---


100%|██████████| 4487/4487 [00:00<00:00, 9647.59it/s]


...

 42%|████▏     | 1867/4487 [00:00<00:00, 9251.03it/s]

--- adding edges ---


100%|██████████| 4487/4487 [00:00<00:00, 9594.25it/s]


--- 100 tweets, 1257 retweeters, 4387 retweets ---


<Figure size 432x288 with 0 Axes>